# 用selenium爬取知网

## 导入模块

In [1]:
import pandas as pd
import numpy as np
from lxml.html import fromstring
import time
from random import random
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 打开驱动，开启网页

In [2]:
opts = webdriver.ChromeOptions()
opts.add_argument('--no-sandbox')#解决DevToolsActivePort文件不存在的报错
opts.add_argument('window-size=1920x3000') #指定浏览器分辨率
opts.add_argument('--disable-gpu') #谷歌文档提到需要加上一这个属性来规避bug
opts.add_argument('--hide-scrollbars') #隐藏滚动条, 应对些特殊页面
#opts.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度
#opts.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败

opts.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" 
# "H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  
driver = webdriver.Chrome( chrome_options = opts)

C:\Users\liwen\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


## 打开知网页面（校外登陆）

In [3]:
# 打开网页
driver.get("https://fsso.cnki.net")

## 准备参数

In [4]:
payload =  {"school": "中山大学南方学院","username":"181025070","password":"01051620"}

In [5]:

query = "((TI=城市矿产+城市矿山+都市矿产+回收企业+回收网络+WEEE+手机回收+电子废弃物+固体废弃物*城市$3+循环经济*城市$3) or (KY=城市矿产+城市矿山+都市矿产+回收企业+回收网络+WEEE+手机回收+电子废弃物+固体废弃物*城市$3+循环经济城市$3) or (SU=城市矿产+城市矿山+都市矿产+回收企业+回收网络+WEEE+手机回收+电子废弃物+固体废弃物*城市$3+循环经济*城市$3))"


## 清空搜索框文本并输入我们准备好的参数

In [6]:
# 清空文本
driver.find_element_by_xpath('//div[@class="submit_input"]//input[@id="o"]').clear()

# 模拟按键输入
driver.find_element_by_xpath('//div[@class="submit_input"]//input[@id="o"]').send_keys(payload['school'])

In [7]:
driver.find_element_by_xpath('//div[@class="submit_button"]').click()

## 清空文本 输入账号密码

In [8]:

driver.find_element_by_xpath('//div[@class="form-element-wrapper"]//input[@name="j_username"]').clear()
driver.find_element_by_xpath('//div[@class="form-element-wrapper"]//input[@name="j_username"]').send_keys(payload['username'])

driver.find_element_by_xpath('//div[@class="form-element-wrapper"]//input[@name="j_password"]').clear()
driver.find_element_by_xpath('//div[@class="form-element-wrapper"]//input[@name="j_password"]').send_keys(payload['password'])

In [9]:
# 登陆
driver.find_element_by_xpath('//div[@class="form-element-wrapper"]/button[@name="_eventId_proceed"]').click()

## 点击接受

In [10]:

driver.find_element_by_xpath('//input[@name="_eventId_proceed"]').click()

## 进入高级检索

In [11]:

driver.find_element_by_xpath('//div[@class="readvce"]/a[@id="highSearch"]').click()

## 查询窗口

In [12]:
# selenium进入新窗口后要切换窗口到你想要的页面

print(driver.window_handles)

['CDwindow-ACDE9653F8E6FADE49E55464419626C4', 'CDwindow-C0078BBCB9C2B80EE301ACE51D56E651']


In [13]:
# 切换窗口
driver.switch_to.window(driver.window_handles[1])

## 进入专业检索

In [14]:
driver.find_element_by_xpath('//dt[@id="searchmenu"]/ul/li[@id="1_3"]/a').click()

## 输入关键词进行检索

In [15]:
# 清空内容
driver.find_element_by_xpath('//dd[@class="dd01"]//textarea[@id="expertvalue"]').clear()

# 输入我们准备好的参数
driver.find_element_by_xpath('//dd[@class="dd01"]//textarea[@id="expertvalue"]').send_keys(query)



In [16]:
# 点击检索
driver.find_element_by_xpath('//span[@class="btnPlace2"]/input[@id="btnSearch"]').click()

## 选择中文文献并每页显示50篇
* 需注意 这里有嵌套HTML

In [17]:
driver.switch_to.default_content();

In [18]:
driver.switch_to.frame("iframeResult")

In [19]:
# 选择中文文献
driver.find_element_by_xpath('//a[text()="中文文献"]').click()

In [20]:
# 每页显示50篇
driver.find_element_by_xpath('//div[@id="id_grid_display_num"]/a[text()="50"]').click()

In [21]:
# 按发表时间排序
driver.find_element_by_xpath('//span[@class="Btn5"]/a[text()="发表时间"]').click()

## 检查时间，从最早的开始收集

In [22]:
element = driver.find_element_by_xpath('//html//table')
main_content = element.get_attribute('innerHTML')
# print(main_content)
pd.read_html(main_content)[3]

,0,1,2,3,4,5,6,7,8,9
0,NaN,题名,作者,来源,发表时间,数据库,被引,下载,阅读,收藏
1,1.0,试论如何提高金属回收企业的经济效益,简敏;,金属回收,1982-07-30,期刊,NaN,6,NaN,收藏
2,2.0,金属回收企业经济效益探讨,刘玉川,金属回收,1983-05-15,期刊,NaN,36,NaN,收藏
3,3.0,一种新型的范德普尔同步振荡器,VASIL UZVNOGLU;陈正良;尤婉贞;马德荣;,电讯技术,1984-10-27,期刊,NaN,48,NaN,收藏
4,4.0,试论“城市矿山”的开发,杨显万; 黎锡辉,云南冶金,1985-04-01,期刊,9,218,NaN,收藏
5,5.0,热回收网络技术改造,蒋金魁,化工设计通讯,1987-04-02,期刊,NaN,5,NaN,收藏
6,6.0,热回收网络优化综合的专家系统,陈丙珍;沈静珠;何小荣;李有润;胡山鹰,石油炼制与化工,1988-03-01,期刊,19,66,NaN,收藏
7,7.0,试论金属回收企业的经济承包,蒋周友;,金属再生,1988-03-01,期刊,NaN,2,NaN,收藏
8,8.0,赞回收企业与钢铁企业的横向联合,李平定,金属再生,1988-05-30,期刊,NaN,9,NaN,收藏
9,9.0,在激流中拼搏的人们——记山东省金属回收公司济南回收站,葛继伟,金属再生,1988-05-30,期刊,NaN,3,NaN,收藏


## 页面解析

In [23]:
element = driver.find_element_by_xpath('//html//table[@class="GridTableContent"]')
main_content = element.get_attribute('outerHTML')
# print(main_content)
pd.read_html(main_content,header=0)[0]

,Unnamed: 0,题名,作者,来源,发表时间,数据库,被引,下载,阅读,收藏
0,1,试论如何提高金属回收企业的经济效益,简敏;,金属回收,1982-07-30,期刊,NaN,6,NaN,收藏
1,2,金属回收企业经济效益探讨,刘玉川,金属回收,1983-05-15,期刊,NaN,36,NaN,收藏
2,3,一种新型的范德普尔同步振荡器,VASIL UZVNOGLU;陈正良;尤婉贞;马德荣;,电讯技术,1984-10-27,期刊,NaN,48,NaN,收藏
3,4,试论“城市矿山”的开发,杨显万; 黎锡辉,云南冶金,1985-04-01,期刊,9.0,218,NaN,收藏
4,5,热回收网络技术改造,蒋金魁,化工设计通讯,1987-04-02,期刊,NaN,5,NaN,收藏
5,6,热回收网络优化综合的专家系统,陈丙珍;沈静珠;何小荣;李有润;胡山鹰,石油炼制与化工,1988-03-01,期刊,19.0,66,NaN,收藏
6,7,试论金属回收企业的经济承包,蒋周友;,金属再生,1988-03-01,期刊,NaN,2,NaN,收藏
7,8,赞回收企业与钢铁企业的横向联合,李平定,金属再生,1988-05-30,期刊,NaN,9,NaN,收藏
8,9,在激流中拼搏的人们——记山东省金属回收公司济南回收站,葛继伟,金属再生,1988-05-30,期刊,NaN,3,NaN,收藏
9,10,回收企业与钢厂横向联合的尝试,宋国平; 王勇,金属再生,1989-01-31,期刊,NaN,3,NaN,收藏


In [24]:
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
main_content

'<html xmlns="http://www.w3.org/1999/xhtml"><head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <title>概览页</title>\n    <meta content="no-cache" http-equiv="pragma">\n     <meta content="always" name="referrer">\n    <link rel="stylesheet" type="text/css" href="//piccache.cnki.net/kdn/nvsmkns/css/gb_min/BriefComp.min.css?v=">\n    <script type="text/javascript" async="" defer="" src="//track.cnki.net/tracking/scripts/piwik.js"></script><script type="text/javascript" src="//piccache.cnki.net/kdn/nvsmkns/script/jquery-1.4.2.min.js"></script>\n    <script type="text/javascript" src="//piccache.cnki.net/kdn/nvsmkns/script/min/gb.BriefPage.min.js?v=D59787997F3B8FCE"></script>\n    \n<script type="text/javascript" src="/KRS/scripts/Recommend.js"></script><script type="text/javascript" src="//piccache.cnki.net/kdn/nvsmkns/script/piwikCommon70.js"></script></head>\n<body leftmargin="0" onload="BriefOnload();" topmargin="0" onkeydown="grabEvent(event);" marginhe

In [25]:
import requests_html  # 比較好上手
html_load = main_content
html = requests_html.HTML(html= html_load, url='https://localhost/')

In [26]:
html.xpath('//html//table[@class="pageBar_bottom"]//a/@href')

['?curpage=2&RecordsPerPage=50&QueryID=12&ID=&turnpage=1&tpagemode=L&dbPrefix=CFLS&Fields=&DisplayMode=listmode&SortType=(%e5%8f%91%e8%a1%a8%e6%97%b6%e9%97%b4%2c%27TIME%27)&PageName=ASP.brief_result_aspx&isinEn=1&',
 '?curpage=3&RecordsPerPage=50&QueryID=12&ID=&turnpage=1&tpagemode=L&dbPrefix=CFLS&Fields=&DisplayMode=listmode&SortType=(%e5%8f%91%e8%a1%a8%e6%97%b6%e9%97%b4%2c%27TIME%27)&PageName=ASP.brief_result_aspx&isinEn=1&',
 '?curpage=4&RecordsPerPage=50&QueryID=12&ID=&turnpage=1&tpagemode=L&dbPrefix=CFLS&Fields=&DisplayMode=listmode&SortType=(%e5%8f%91%e8%a1%a8%e6%97%b6%e9%97%b4%2c%27TIME%27)&PageName=ASP.brief_result_aspx&isinEn=1&',
 '?curpage=5&RecordsPerPage=50&QueryID=12&ID=&turnpage=1&tpagemode=L&dbPrefix=CFLS&Fields=&DisplayMode=listmode&SortType=(%e5%8f%91%e8%a1%a8%e6%97%b6%e9%97%b4%2c%27TIME%27)&PageName=ASP.brief_result_aspx&isinEn=1&',
 '?curpage=6&RecordsPerPage=50&QueryID=12&ID=&turnpage=1&tpagemode=L&dbPrefix=CFLS&Fields=&DisplayMode=listmode&SortType=(%e5%8f%91%e8%a

In [27]:
html.xpath('//html//table[@class="pageBar_bottom"]//a/text()')

['2', '3', '4', '5', '6', '7', '8', '9', '下一页']

## 测试循环页码

In [29]:
# 测试最大页码
# math.ceil 向上取整
import requests_html
import math
elemente = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')
html = requests_html.HTML(html= main_content, url='https://localhost/')
content = html.xpath('//*[@id="ctl00"]/table/tbody/tr[3]/td/table/tbody/tr/td/div/div')
max_文章数量 =  str(content[0].lxml.text_content()).split()[1].replace(',','')
print(max_文章数量)
max_文章页码 = math.ceil((float(max_文章数量)/50))
print("max_文章页码:",max_文章页码,'\n',type(max_文章页码))

5783
max_文章页码: 116 
 <class 'int'>


## 勾选本页文献

In [29]:
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
driver.find_element_by_xpath('//input[@id="selectCheckbox"]').click()

## 检查勾选文献数量

In [30]:
# 因为页面更新，html重新用driver获取一次
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
html = requests_html.HTML(html= main_content, url='https://localhost/')
已选文献 = html.xpath('//*[@id="J_ORDER"]/tbody/tr[2]/td/table/tbody/tr/td[1]/div/span/em')[0].lxml.text_content()
已选文献

'50'

## 翻页

In [31]:
# 下一页
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML') 
a = driver.find_element_by_xpath('//a[text()="下一页"]')
a.click()

## 清除已选文献

In [58]:
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML') 
a = driver.find_element_by_xpath('//a[text()="清除"]').click()
a

## 导出文献

In [61]:
# 回到第一页勾选文献
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
a = driver.find_element_by_xpath('//*[@id="ctl00"]/table/tbody/tr[3]/td/table/tbody/tr/td/div/a[3]')
a.click()

driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
driver.find_element_by_xpath('//input[@id="selectCheckbox"]').click()

In [62]:
# 导出文献
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML') 
a = driver.find_element_by_xpath('//a[text()=" 导出/参考文献"]')
a.click()

## 测试：回到首页

In [1]:
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
a = driver.find_element_by_xpath('//a[text()="首页"]')
a.click()

## 循环页码

In [28]:
import math  # 向上取整
import time
from random import random
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML')  
html = requests_html.HTML(html= main_content, url='https://localhost/')
content = html.xpath('//*[@id="ctl00"]/table/tbody/tr[3]/td/table/tbody/tr/td/div/div')
max_文章数量 =  str(content[0].lxml.text_content()).split()[1].replace(',','')
max_文章页码 = math.ceil((float(max_文章数量)/50))


for page in range(0,max_文章页码):
    # 检查数量
    element = driver.find_element_by_xpath('//html')
    main_content =element.get_attribute('outerHTML')  
    html = requests_html.HTML(html= main_content, url='https://localhost/')
    已选文献 = html.xpath('//*[@id="J_ORDER"]/tbody/tr[2]/td/table/tbody/tr/td[1]/div/span/em')[0].lxml.text_content()
    print(int(已选文献))
    if int(已选文献) < 500:
        # 勾选本页文献
        driver.find_element_by_xpath('//input[@id="selectCheckbox"]').click()
        # 等待时间 平均时间38*103
        time.sleep(3)
        # 翻页
        driver.switch_to.default_content()
        driver.switch_to.frame("iframeResult")
        element = driver.find_element_by_xpath('//html')
        main_content =element.get_attribute('outerHTML')  
        main_content
        a = driver.find_element_by_xpath('//a[text()="下一页"]')
        a.click()

    else:
        # 进入导出/参考文献  页面（不同窗口）
        driver.switch_to.default_content()
        driver.switch_to.frame("iframeResult")
        element = driver.find_element_by_xpath('//html')
        main_content =element.get_attribute('outerHTML') 
        main_content
        a = driver.find_element_by_xpath('//a[text()=" 导出/参考文献"]')
        a.click()
# ----------------------------
# 缺下载Refworks操作
# ----------------------------
        # 清除已选文章，继续翻页
        # 回到cnki搜索文章页面（切换窗口）
        driver.switch_to.window(driver.window_handles[1])
        # 清除已选
        driver.switch_to.default_content()
        driver.switch_to.frame("iframeResult")
        element = driver.find_element_by_xpath('//html')
        main_content =element.get_attribute('outerHTML') 
        a = driver.find_element_by_xpath('//a[text()="清除"]').click()
        

0
50
100
150
200
250
300
350
400
450
500
0
50
100
150
200


IndexError: list index out of range

In [29]:
from PIL import Image
import os
import os.path
import time
from lxml.html import fromstring
import datetime
from glob import glob
import  base64
import requests
import json

## 验证码
### 手动输入

In [33]:
ele = driver.find_element_by_id("CheckCodeImg")
ele.screenshot(".\验证码图\crake code.png")
Image.open(".\验证码图\crake code.png")

In [37]:
crake_code = input("输入验证码：")

输入验证码：2T0DR


In [38]:
crake_code

'2T0DR'

In [39]:
driver.find_element_by_xpath('//input[@id="CheckCode"]').send_keys(crake_code)
driver.find_element_by_xpath('//input[@value="提交"]').click()

### 使用百度API

In [30]:
pic_url = driver.find_element_by_xpath('//*[@id="CheckCodeImg"]').get_attribute('src')

In [31]:
import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=q7PWZBq1nltWsUMNtmGNomMz&client_secret=gyECcjGtbU7YOBrCX8o9s1zKaHjGkP8R'
response = requests.get(host)
if response:
    print(response.json())

{'refresh_token': '25.863295379b572afc45b0d3d919f45ae0.315360000.1907209739.282335-20313380', 'expires_in': 2592000, 'session_key': '9mzdCr3ZA9tFh2zNalqX0bplFzpwAx0reA60v5LtChCYgObxs2VZNIx2dRRVT3OO9QOYBV8Xny+Goi7McGtvpDB7fW7dIA==', 'access_token': '24.193eb610103702b523b41af89bd72be3.2592000.1594441739.282335-20313380', 'scope': 'public vis-ocr_ocr brain_ocr_scope brain_ocr_general brain_ocr_general_basic vis-ocr_business_license brain_ocr_webimage brain_all_scope brain_ocr_idcard brain_ocr_driving_license brain_ocr_vehicle_license vis-ocr_plate_number brain_solution brain_ocr_plate_number brain_ocr_accurate brain_ocr_accurate_basic brain_ocr_receipt brain_ocr_business_license brain_solution_iocr brain_qrcode brain_ocr_handwriting brain_ocr_passport brain_ocr_vat_invoice brain_numbers brain_ocr_business_card brain_ocr_train_ticket brain_ocr_taxi_receipt vis-ocr_household_register vis-ocr_vis-classify_birth_certificate vis-ocr_台湾通行证 vis-ocr_港澳通行证 vis-ocr_机动车购车发票识别 vis-ocr_机动车检验合格证识别 vis

In [32]:
a = response.json()
access_token = a["access_token"]
access_token

'24.193eb610103702b523b41af89bd72be3.2592000.1594441739.282335-20313380'

In [34]:
import requests
import base64

'''
通用文字识别
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic"
# 二进制方式打开图片文件
f = open('.\验证码图\crake code.png', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
access_token = access_token
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())

{'log_id': 1334492320820211883, 'words_result_num': 1, 'words_result': [{'words': '02640'}]}


In [36]:
b=response.json()
code = b['words_result'][0]['words']
code

'02640'

In [37]:
driver.find_element_by_xpath('//input[@id="CheckCode"]').send_keys(code)
driver.find_element_by_xpath('//input[@value="提交"]').click()

## 导出文献

In [38]:
driver.switch_to.default_content()
driver.switch_to.frame("iframeResult")
element = driver.find_element_by_xpath('//html')
main_content =element.get_attribute('outerHTML') 
a = driver.find_element_by_xpath('//a[text()=" 导出/参考文献"]')
a.click()

In [39]:
# 切换窗口
driver.switch_to.window(driver.window_handles[-1])

In [40]:
driver.find_element_by_xpath('//input[@id="exportExcel"]').click()